In [64]:
# Input data.
# Since the classes are unequal, we don't risk random sampling from full set.
# Instead, keep them separate until we make the train/test split.
infile_pos='score-positive.4mer.features.csv'
infile_neg='score-negative.4mer.features.csv'
infile_zero='score-zero.4mer.features.csv'
import numpy as np
import pandas as pd
raw_pos = pd.read_csv(infile_pos,header=0)
raw_neg = pd.read_csv(infile_neg,header=0)
raw_zero = pd.read_csv(infile_zero,header=0)
raw_pos.head()

,label,seqname,AAAA,AAAC,AAAG,AAAT,AACA,AACC,AACG,AACT,...,TTCG,TTCT,TTGA,TTGC,TTGG,TTGT,TTTA,TTTC,TTTG,TTTT
0,1,ENSG00000261061,31,5,2,13,6,5,0,5,...,0,2,5,5,2,4,2,3,4,7
1,1,ENSG00000254837,87,19,28,42,31,9,1,15,...,1,15,13,13,6,14,17,11,10,30
2,1,ENSG00000282851,105,29,43,51,48,23,2,23,...,3,21,21,20,14,27,28,23,22,62
3,1,ENSG00000255650,108,32,51,55,55,25,3,26,...,3,24,21,21,19,30,28,28,25,83
4,1,ENSG00000163597,133,47,74,80,65,39,8,42,...,6,45,37,32,37,61,66,55,51,169


In [67]:
def split_train_test(dataset,train_portion):    
    # Use pandas sample() to randomize the order (i.e. random sample without replacement).
    # Side note. Data frame can be randomized while keeping header in place.
    # This is repeatable with 42.
    middle = int(len(dataset)*train_portion)
    random = dataset.sample(frac=1,random_state=42)
    train_set = random[:middle]
    test_set = random[middle:]
    return train_set,test_set
TRAIN_PORTION=0.8

train_pos , test_pos = split_train_test(raw_pos,TRAIN_PORTION)
train_neg , test_neg = split_train_test(raw_neg,TRAIN_PORTION)
train_zero , test_zero = split_train_test(raw_zero,TRAIN_PORTION)

raw_pos.shape, train_pos.shape, test_pos.shape

((170, 258), (136, 258), (34, 258))

In [53]:
# The Geron book recommends this (or StrattifiedShuffleSplit) 
# but we could not make it work. 

#from sklearn.model_selection import ShuffleSplit
#splitter=ShuffleSplit(n_splits=1, train_size=0.8, random_state=42)
#for train_index,test_index in splitter.split(rawpos):
#    train_pos = rawpos[train_index]
#    test_pos = rawpos[test_index]

In [69]:
# Combine and shuffle.
train_sorted = train_pos + train_neg + train_zero
train_set = train_sorted.sample(frac=1,random_state=42)

test_sorted = test_pos + test_neg + test_zero
test_set = test_sorted.sample(frac=1,random_state=42)

train_set.shape, test_set.shape

((504, 258), (164, 258))

,-1,ENSG00000259891,71,8,9,16,16.1,5,0,1,...,2.14,9.21,20,11.13,8.15,8.16,11.14,3.20,19.1,11.15
202,-1,ENSG00000275441,4896,2145,2659,3019,2222,1593,446,1692,...,468,2887,1839,1720,2221,2186,2483,2834,2785,5328
330,-1,ENSG00000271918,7846,3503,4499,4827,3627,2657,730,2900,...,765,4807,3039,2843,3645,3504,4020,4692,4564,8459
24,-1,ENSG00000267546,933,369,456,554,362,247,68,306,...,62,474,330,252,300,384,445,499,452,1051
992,0,ENSG00000255121,7011,3012,3865,4763,3170,2064,548,2581,...,469,4159,2749,2302,2844,3072,3927,4023,4050,7590
664,1,ENSG00000278231,2064,908,1162,1352,981,564,157,738,...,128,1081,740,627,804,841,1011,1023,1039,2004
